<a href="https://colab.research.google.com/github/Buuuuli/AIPI540_individual/blob/main/pretrain_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls

drive  sample_data


In [ ]:
%cd /content/drive/"My Drive"/AIPI540_NLP/data

/content/drive/My Drive/AIPI540_NLP/data


In [ ]:


import math
import os
import numpy as np
import pandas as pd
import string
import time
import copy
from sklearn.linear_model import LogisticRegression
import urllib.request
import zipfile
from torch.utils.tensorboard import SummaryWriter
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader,TensorDataset
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch import nn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer
import sentencepiece
import tensorflow as tf

from transformers import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
% matplotlib inline


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [ ]:
data = pd.read_parquet("full_raw_data.parquet.gzip")
print(data.count())

sentiment    1236654
text         1236654
dtype: int64


In [ ]:
data['sentiment'] = data['sentiment'].map({"neutral":1,"positive":2,"negative":0})

In [ ]:
# Divide training data by class
df_class0 = data[data['sentiment'] == 0]
df_class1 = data[data['sentiment'] == 1]
df_class2 = data[data['sentiment'] == 2]

In [ ]:
# Count the number in each class
count_class0 = (data['sentiment'] ==0 ).sum()
count_class1 = (data['sentiment'] ==1 ).sum()
count_class2 = (data['sentiment'] ==2 ).sum()

In [ ]:
# Resample class 0 so count is equal to class 1
df_class2_under = df_class2.sample(n=count_class0)
df_class1_under = df_class1.sample(n=count_class0)
# Add the undersampled class 0 together with class 1
df_undersampled = pd.concat([df_class2_under,df_class1_under,df_class0])

In [ ]:
df_undersampled.sentiment.value_counts()

2    117142
1    117142
0    117142
Name: sentiment, dtype: int64

In [ ]:
data = df_undersampled

In [ ]:
df = data.sample(frac = 0.2)

In [ ]:
len(df)

70285

In [ ]:
data = df

In [ ]:
# Get sentence data
sentences = data.text.values
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]


In [ ]:
# Get tag labels data
labels = data.sentiment.values
print(labels[0])

1


In [ ]:
sentences[0]

"The movie picks up after 30 minutes in. I think it's because of the character JR. This woman sparks up most of the interesting scenes in this movie. What I like about this movie is that in has a lot of real women. They aren't the Hollywood standard of beauty, but they are beautiful. They are like the type of beautiful you can find next door, or down the street rather than in a magazine. Another good thing about this movie is that there is a diversity in the lesbians. There are butch, soft butch, femme, lipstick lesbian, straight girl that becomes gay, tomboy, etc. Fun fun. And some of the material that explores lesbian relationships is fairly accurate.I wouldn't buy this movie, however, unless you're into buying *bad* films that you can laugh at.*The worst part of this movie is the plot.*It's way underdeveloped. The first half hour was completely unnecessarily. I think the main reason the plot is unsuccessful is because the movie tries to fit in too many aspects of lesbian life and re

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in tqdm(sentences)]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 70285/70285 [01:39<00:00, 704.53it/s]

Tokenize the first sentence:
['▁the', '▁movie', '▁picks', '▁up', '▁after', '▁30', '▁minutes', '▁in', '.', '▁', 'i', '▁think', '▁it', "'", 's', '▁because', '▁of', '▁the', '▁character', '▁', 'j', 'r', '.', '▁this', '▁woman', '▁spark', 's', '▁up', '▁most', '▁of', '▁the', '▁interesting', '▁scenes', '▁in', '▁this', '▁movie', '.', '▁what', '▁', 'i', '▁like', '▁about', '▁this', '▁movie', '▁is', '▁that', '▁in', '▁has', '▁a', '▁lot', '▁of', '▁real', '▁women', '.', '▁they', '▁aren', "'", 't', '▁the', '▁', 'hollywood', '▁standard', '▁of', '▁beauty', ',', '▁but', '▁they', '▁are', '▁beautiful', '.', '▁they', '▁are', '▁like', '▁the', '▁type', '▁of', '▁beautiful', '▁you', '▁can', '▁find', '▁next', '▁door', ',', '▁or', '▁down', '▁the', '▁street', '▁rather', '▁than', '▁in', '▁a', '▁magazine', '.', '▁another', '▁good', '▁thing', '▁about', '▁this', '▁movie', '▁is', '▁that', '▁there', '▁is', '▁a', '▁diversity', '▁in', '▁the', '▁lesbian', 's', '.', '▁there', '▁are', '▁but', 'ch', ',', '▁soft', '▁but', 'ch'

In [ ]:
MAX_LEN = 128

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tqdm(tokenized_texts)]

100%|██████████| 70285/70285 [00:21<00:00, 3338.19it/s]


In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in tqdm(input_ids):
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

100%|██████████| 70285/70285 [00:04<00:00, 14381.16it/s]


In [ ]:
input_ids

array([[   18,  1432, 12491, ...,    19,  1632,  1615],
       [   17,   150,   675, ...,    17,  4190, 13323],
       [   52,    51,    22, ..., 15069,  2001,    11],
       ...,
       [   52, 11968,  2197, ...,    36,   288,    25],
       [   25,    18,   544, ...,   368,  1372,     9],
       [   17, 13304, 14500, ...,     0,     0,     0]])

In [ ]:
#Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=56, test_size=0.2)

In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=3)
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in tqdm(enumerate(train_dataloader)):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # # Validation

  # # Put model in evaluation mode to evaluate loss on the validation set
  # model.eval()

  # # Tracking variables 
  # eval_loss, eval_accuracy = 0, 0
  # nb_eval_steps, nb_eval_examples = 0, 0

  # # Evaluate data for one epoch
  # for batch in validation_dataloader:
  #   # Add batch to GPU
  #   batch = tuple(t.to(device) for t in batch)
  #   # Unpack the inputs from our dataloader
  #   b_input_ids, b_input_mask, b_labels = batch
  #   # Telling the model not to compute or store gradients, saving memory and speeding up validation
  #   with torch.no_grad():
  #     # Forward pass, calculate logit predictions
  #     output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  #     logits = output[0]
    
  #   # Move logits and labels to CPU
  #   logits = logits.detach().cpu().numpy()
  #   label_ids = b_labels.to('cpu').numpy()

  #   tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
  #   eval_accuracy += tmp_eval_accuracy
  #   nb_eval_steps += 1

  # print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.22it/s]
2it [00:00,  3.36it/s]
3it [00:00,  3.41it/s]
4it [00:01,  3.43it/s]
5it [00:01,  3.45it/s]
6it [00:01,  3.46it/s]
7it [00:02,  3.46it/s]
8it [00:02,  3.43it/s]
9it [00:02,  3.44it/s]
10it [00:02,  3.45it/s]
11it [00:03,  3.45it/s]
12it [00:03,  3.46it/s]
13it [00:03,  3.45it/s]
14it [00:04,  3.46it/s]
15it [00:04,  3.46it/s]
16it [00:04,  3.45it/s]
17it [00:04,  3.46it/s]
18it [00:05,  3.44it/s]
19it [00:05,  3.45it/s]
20it [00:05,  3.46it/s]
21it [00:06,  3.45it/s]
22it [00:06,  3.46it/s]
23it [00:06,  3.46it/s]
24it [00:06,  3.46it/s]
25it [00:07,  3.46it/s]
26it [00:07,  3.46it/s]
27it [00:07,  3.44it/s]
28it [00:08,  3.44it/s]
29it [00:08,  3.43it/s]
30it [00:08,  3.44it/s]
31it [00:09,  3.44it/s]
32it [00:09,  3.45it/s]
33it [00:09,  3.41it/s]
34it [00:09,  3.42it/s]
35it [00:10,  3.42it/s]
36it [00:10,  3.42it/s]
37it [00:10,  3.42it/s]
38it [00:11,  3.43it/s]
39it [00:11,  3.42it/s]
40it [00:

Train loss: 0.5270050162019312



0it [00:00, ?it/s]
1it [00:00,  3.39it/s]
2it [00:00,  3.42it/s]
3it [00:00,  3.41it/s]
4it [00:01,  3.41it/s]
5it [00:01,  3.41it/s]
6it [00:01,  3.40it/s]
7it [00:02,  3.41it/s]
8it [00:02,  3.41it/s]
9it [00:02,  3.41it/s]
10it [00:02,  3.42it/s]
11it [00:03,  3.41it/s]
12it [00:03,  3.39it/s]
13it [00:03,  3.40it/s]
14it [00:04,  3.41it/s]
15it [00:04,  3.41it/s]
16it [00:04,  3.41it/s]
17it [00:04,  3.41it/s]
18it [00:05,  3.42it/s]
19it [00:05,  3.42it/s]
20it [00:05,  3.42it/s]
21it [00:06,  3.42it/s]
22it [00:06,  3.42it/s]
23it [00:06,  3.42it/s]
24it [00:07,  3.42it/s]
25it [00:07,  3.42it/s]
26it [00:07,  3.41it/s]
27it [00:07,  3.42it/s]
28it [00:08,  3.42it/s]
29it [00:08,  3.42it/s]
30it [00:08,  3.40it/s]
31it [00:09,  3.41it/s]
32it [00:09,  3.41it/s]
33it [00:09,  3.41it/s]
34it [00:09,  3.41it/s]
35it [00:10,  3.41it/s]
36it [00:10,  3.41it/s]
37it [00:10,  3.41it/s]
38it [00:11,  3.41it/s]
39it [00:11,  3.42it/s]
40it [00:11,  3.42it/s]
41it [00:12,  3.42it/s]
42it 

Train loss: 0.395479399261655


In [ ]:
%cd /content/drive/"My Drive"/AIPI540_NLP

/content/drive/My Drive/AIPI540_NLP


In [ ]:
torch.save(model.state_dict(),'/model_1percent.pt')

In [ ]:
filename = 'pretrained2.pt'

torch.save(model, filename)

In [ ]:
!ls

clean_text.csv	full_raw_data.parquet.gzip  runs


In [ ]:
# torch.save(model.state_dict(), directory_path+'/model_without_language_model.ckpt')
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [ ]:
print('Test Accuracy of the model on vla data is: {} %'.format(100 * correct / total))

Test Accuracy of the model on vla data is: 79.86056768869602 %
